In [114]:
# Import necessary libraries
import base64
import json
import sys 
import os
import fitz
import requests
import re
from fitz import Document, Page, Rect

# Load double page

In [115]:
src = fitz.open("../PyMuPDF_playground/574_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(5)



# Extract category of double page

Create list of categories

In [116]:
categories = ['editorial', 'augustiner:in', 'einsicht', 'das wahre leben', 'cover', 'tun & lassen', 'vorstadt', 'lokalmatador:in nº', 'vorstadt', 'art.ist.in', 'dichter innenteil', 'augustinchen']

If same category, do not split into single pages.
If different categories, split couble page into two single pages

In [117]:
src = fitz.open("../PyMuPDF_playground/574_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(2)


rect = fitz.Rect(0, 0, page.rect.width, 60)
text = page.get_textbox(rect)
# Only remove all digits if lokalmatador is not in the text
if 'lokalmatador:in' not in text:
    # Remove all digits from the text
    result = ''.join(i for i in text if not i.isdigit())
    # Remove all \n from the text
    #result = result.replace(' ', '')
    result = result.lower()
    print(result)
else:
    char_index_beginning = text.find('lokalmatador:in')
    char_index_end = char_index_beginning + 22
    lokalmatadorin = text[char_index_beginning:char_index_end]
    text = text.replace(lokalmatadorin, '')
    # Remove all digits from the text
    result = ''.join(i for i in text if not i.isdigit())
    # Remove all \n from the text
    #result = result.replace(' ', '')
    result = result.lower()
    print(result)
i = 0
for word in result.split('\n'):
    word = word.strip() 
    if word in categories:
        i += 1

if i == 2:
    print('Both categories on top')
elif i == 1:
    print('One category on top')
    # Execute code below to extract category from left side
else:
    raise Exception('No category on top')





das wahre leben 

einsicht
Both categories on top


If in first print statement only one category has been found, check if category is written vertically on outer left side of page 

In [118]:
rect = fitz.Rect(0, 0, 50, page.rect.height)
text = page.get_text("rect", rect)
# Remove all digits from the text
result = ''.join(i for i in text if not i.isdigit())
# Remove all \n from the text
result = result.replace('\n', '')
result = result.lower()
result

'soscgm'

# Extract text from double page

In [169]:
src = fitz.open("../PyMuPDF_playground/574_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(3)
# Check for chars in different color than black and font size bigger than 50
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

main_title = sub_title = excerpt = author = illustrator = photo_credits = quotes = original_photo_text = original_illustrator_text = original_author_text = starting_chars = ""
font_size = 0

# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = page.get_text("dict", flags=11)["blocks"]
for b in blocks:  # iterate through the text blocks
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            font_properties = "Font: '%s' (%s), size %g, color #%06x" % (
                s["font"],  # font name
                flags_decomposer(s["flags"]),  # readable font flags
                s["size"],  # font size
                s["color"],  # font color
            )
            # print("Text: '%s'" % s["text"])  # simple print of text
            # print(font_properties)
            # if case to search for a text start 
            colour_code = "#%06x" % (s["color"])
            if colour_code != "#2e2013" and colour_code != "#000000":
                if not s["text"].isdigit() and len(s["text"]) < 3 and s["text"] != "■":
                    starting_chars = s["text"]
            
            # TODO search for the word before the "■"
            # if case to search for a text end with "■"
            # if colour_code != "#2e2013" and colour_code != "#000000" and s["text"] == "■":
            #     ending_words = s["text"]
            #     print("End: '%s'" % s["text"], s["color"])  # simple print of text

            # if case to search for a title
            # if s["flags"] & 2 ** 4:
            #     print("Title: '%s'" % s["text"], font_properties)

            if s["font"] == "AmasisMTStd-Bold":
                # Fill main_title with first line of main title
                if main_title == "" and font_size == 0:
                    main_title = s["text"]
                    font_size = s["size"]
                else:
                    # Check if next line belongs to main title
                    if s["size"] == font_size:
                        main_title += s["text"]
                    # This means that the next line is a subtitle
                    else:
                        sub_title = s["text"]
            
            # This snippet is not for the cover page but for the following pages like "tun&lassen"
            # if s["font"] == "Sentinel-Medium":
            #     print("Second Title of Excerpt: '%s'" % s["text"], font_properties)

            if s["font"] == "MyriadPro-Regular" and s["text"] not in categories:
                if "text" in s["text"].lower():
                    original_author_text = s["text"]
                    author = s["text"].split(":")[1].upper()
                elif "illustration" in s["text"].lower():
                    original_illustrator_text = s["text"]
                    illustrator = s["text"].split(":")[1]
                    illustrator = illustrator.strip().upper()
                elif "foto" in s["text"].lower():
                    original_photo_text = s["text"]
                    photo_credits = s["text"].split(":")[1].upper()
                else:
                    excerpt += s["text"]
                
               
            # if case to search for quotes 
            # TODO how to handle author name below the quote written in bold
            if s["flags"] & 2 ** 1 and s["font"] == "AmasisMTStd-Italic" and s["size"] == 17:
                quotes += s["text"]

print("Main Title:", main_title, sub_title)
print("Excerpt:", excerpt)
print("Credits:", author, illustrator, photo_credits)
print("Quotes:", quotes)
print("Starting Chars:", starting_chars)

def determine(line):
    # Remove numbers from text
    if line.isdigit():
        return False
    # Remove category cover from text
    elif "cover" in line.lower():
        return False
    else:
        return True

def find_starting_and_ending_point_of_text(article):
    article = list(article)
    article_edit = article
    print(article)
    for index, letter in enumerate(article):
        print("letter", letter, index)
        if len(starting_chars) == 1:
            if letter == starting_chars:
                print("found starting char")
                del article_edit[:index-1]
        elif len(starting_chars) == 2:
            if letter == starting_chars[0] and article[index + 1] == starting_chars[1] and article[index + 2] == "\n":
                print("found starting chars")
                del article_edit[:index-1]
        elif len(starting_chars) == 3:
            if letter == starting_chars[0] and article[index + 1] == starting_chars[1] and article[index + 2] == starting_chars[2] and article[index + 3] == "\n":
                print("found starting chars")
                del article_edit[:index-1]
        else:
            print("No starting chars found")
        
        if letter == "■":
            print("found ending char")
            del article_edit[index:]
            
        print("Find func", article_edit)
        return article_edit


def format_text(article):
    # Format the string
    article = list(article)
    article_edit = article
    for index, letter in enumerate(article):
        # deletes newlines that are made up because of articles structure
        if "\n" in letter:
            if (
                "." in article[index - 1]
                or "!" in article[index - 1]
                or "?" in article[index - 1]
                or ":" in article[index - 1]
            ):
                print("supposed to break a new line", letter)
            elif "-" in article[index - 1]:

                if " " in article[index - 1] and " " in article[index + 1]:
                    # this hyphen has been set on purpose and stays
                    print("hyphen has been set on purpose")
                else:
                    # deletes hyphen on the index and the extra space before it
                    del article_edit[index]
                    del article_edit[index - 1]
            else:
                del article_edit[index]
        # deletes double spaces
        if " " in letter and " " in article[index - 1]:
            del article_edit[index]

    # to create a readable text for the article
    readable_text = ""
    for letter in article_edit:
        readable_text += letter
    
    return readable_text



whole_text = page.get_text("text")

# Find starting and ending point of text
# whole_text = find_starting_and_ending_point_of_text(whole_text)

# Format text
output_text = format_text(whole_text)

# TODO how to delete beginning number and category cover
# whole_text[:] = [tup for tup in whole_text if determine(tup)]

output_text = output_text.replace(main_title, "")
output_text = output_text.replace(sub_title, "")
output_text = output_text.replace(excerpt, "")
if original_author_text != "":
    output_text = output_text.replace(original_author_text, "")
if original_illustrator_text != "":
    output_text = output_text.replace(original_illustrator_text, "")
if original_photo_text != "":
    output_text = output_text.replace(original_photo_text, "")
# TODO how to handle quotes
# output_text = output_text.replace(quotes, "")



Main Title: Demokratisieren wir die Demokratie! Zufall, Los und Mitbestimmung.
Excerpt:  Die repräsentative Demokratie ist nicht in der lage, angemessen auf die Unpässlichkeit des Planeten zu reagieren. Derweil bringen die schulstreikenden 15-Jährigen ihren eltern einen noch wenig gebrauchten Begriff bei: die aleatorische Demokratie.
Credits:  ROBERT SOMMER BERND PEGRITZ 
Quotes: Mich macht zum Beispiel der Fluglärm zornig. Mir geht ein «Bürger:innenrat Flugverkehr» ab
Starting Chars: D
supposed to break a new line 

supposed to break a new line 

supposed to break a new line 

supposed to break a new line 

supposed to break a new line 

supposed to break a new line 



Since there is no end found the text must be extracted and added together from the next page

In [ ]:
src = fitz.open("../PyMuPDF_playground/574_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(4)

## Bild Extrahierung mit Bildbeschreibung

In [120]:
global_path = (
    "/home/funny/Documents/Projects/230701-Parsing-Tool/augustin-plugin/sample_data/"
)

global_url = "http://localhost:10004/wp-json/wp/v2/"

def upload_image_notebook(image_path, image_subheaders):
    """Upload the image to the Wordpress media library."""
    url = global_url + "media"

    user = "lebe"
    password = "gUwM J4pU sngD VHpk Cub7 quS2"

    credentials = user + ":" + password

    token = base64.b64encode(credentials.encode())

    header = {"Authorization": "Basic " + token.decode("utf-8")}

    
    # if case of 0 image_subheaders is not being covered since it is fine if there is no additional meta information
    if (len(image_subheaders) == 1):
        caption = image_subheaders[0]
        print(caption)
        media = {"file": open(image_path, "rb"), "title": caption}
    elif (len(image_subheaders) == 2):
        caption = image_subheaders[0]
        description = image_subheaders[1]
        print("caption,descirption: ",caption, description)
        media = {"file": open(image_path, "rb"), "title": caption}
    else:
        raise Exception("More than 2 image subheaders found", (len(image_subheaders)), image_subheaders)

    response = requests.post(url, headers=header, files=media)

    image_id = json.loads(response.content)["id"]

    return image_id

# src = fitz.open("../PyMuPDF_playground/570_augustin_fertig_kontrolle.pdf")
src = fitz.open("./temp.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(3)

pix = page.get_pixmap()  # render page to an image
pix.save("page-%i.png" % page.number) 

caption_list_for_removing_text = []
items = page.get_images(full=True)
for item in items:
    xref = item[0]
    rect = page.get_image_bbox(item)
    rect.y1 = rect.y1 + 30
    rect.x0 = rect.x0 - 10
    rect.x1 = rect.x1 + 10
    page.draw_rect(rect, color=fitz.utils.getColor('purple'))
    src.save("temp.pdf")
    image_subheaders = page.get_textbox(rect).split("\n")
    print(image_subheaders)
    caption_list_for_removing_text.append(image_subheaders)
    image = src.extract_image(xref)
    image_name = src.name.split(".")[0] + "_image_" + str(xref)
    image_path = os.path.join(
        global_path,
        image_name + "." + image["ext"],
    )
    with open(
        (image_path),
        "wb",
    ) as imgout:
        imgout.write(image["image"])
        imgout.close()
    
    upload_image_notebook(image_path, image_subheaders)


['Das CND-Friedenssymbol (wie hier statt dem «O» in MORE) wurde zum ersten Mal vor 65 Jahren in London hochgehalten', '© nina thiel']
caption,descirption:  Das CND-Friedenssymbol (wie hier statt dem «O» in MORE) wurde zum ersten Mal vor 65 Jahren in London hochgehalten © nina thiel
['Claudia Brunner: Friedensforscherin', '© riccio.at, Klagenfurt']
caption,descirption:  Claudia Brunner: Friedensforscherin © riccio.at, Klagenfurt
['Werner Wintersteiner: Friedensforscher', 'foto: christoph fellmer']
caption,descirption:  Werner Wintersteiner: Friedensforscher foto: christoph fellmer


# Angehensweise zur Extrahierung des Cover-Texts
1. Nimm den gesamten Text von `page.get_text("text")`
2. Bestimme Titel, Ausschnitt, Zitate, Bildunterschriften, Credits (Author, Foto etc.)
3. Lösche schrittweise Passagen aus dem Text, die nicht Teil des Inhalts sind (Titel, Ausschnitt...) -> Inhalt muss zwischen diesen Passagen liegen
4. Abhängig, ob auf der gleichen Seite schon der Text endet durch ein "■", muss der Text mit der zweiten Seite zusammengeknüpft werden
5. Sende das Cover Bild auf der linken Seite als Hauptbild mit und füge die restlich Bilder mit an

## TODO
1. Schicke Bild-caption und -description mit hinzu [Source](https://developer.wordpress.org/rest-api/reference/media/#schema-caption)
2. [Post](https://wordpress.org/support/topic/how-to-include-multiple-images-in-wordpress-wp-api-post/) that says multiple images in REST API Post-Request not possible

# Split double page in two single pages

In [121]:
r = page.rect
d = fitz.Rect(
    # CropBox displacement if being set
    page.cropbox_position,
    page.cropbox_position,
)  # starting at (0, 0)

r1 = r
r1.x1 = r1.x1 / 2  # left side of double page
r2 = r1 + (r1.width, 0, r1.width, 0)  # right side of double page
# r2 = r + (r.width / 2, 0, 0, 0)  # right side of double page

r2 += d  # add the CropBox displacement
new_page_left = new_doc.new_page(-1, width=r1.width, height=r1.height)
new_page_left.show_pdf_page(
    new_page_left.rect,
    src,
    page.number,
    clip=r1,
)

67